In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

%reload_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('data/unclean_data.csv')
df

,Immoweb ID,Property type,property sub-type,Price,Post code,Building condition,Kitchen type,Bedrooms,Furnished,Terrace surface,Tenement building,Number of frontages,Swimming pool,How many fireplaces?,Garden,Terrace,Surface of the plot,Living area,Garden surface,Garden orientation
0,9729720,APARTMENT,PENTHOUSE,179000,1140,Good,Semi equipped,1.0,Yes,9.0,No,NaN,NaN,NaN,NaN,NaN,NaN,63.0,NaN,NaN
1,9729785,APARTMENT,PENTHOUSE,255000,8370,Good,Installed,1.0,Yes,46.0,No,2.0,NaN,NaN,NaN,NaN,NaN,42.0,NaN,NaN
2,9729784,APARTMENT,PENTHOUSE,255000,8370,Good,Installed,1.0,Yes,46.0,No,2.0,NaN,NaN,NaN,NaN,NaN,42.0,NaN,NaN
3,9729780,APARTMENT,PENTHOUSE,620000,8370,Good,Installed,3.0,Yes,70.0,No,2.0,NaN,NaN,NaN,NaN,NaN,150.0,NaN,NaN
4,9727201,APARTMENT,PENTHOUSE,379000,1020,Good,USA hyper equipped,2.0,No,110.0,No,3.0,No,NaN,NaN,NaN,NaN,120.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,APARTMENT,SERVICE_FLAT,132000,1730,As new,NaN,1.0,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14537,7770553,APARTMENT,SERVICE_FLAT,177000,1730,As new,NaN,1.0,NaN,NaN,No,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
14538,7770551,APARTMENT,SERVICE_FLAT,190000,1730,As new,NaN,2.0,NaN,NaN,No,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
14539,7921693,APARTMENT,SERVICE_FLAT,1350000,5600,NaN,NaN,30.0,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,1500.0,NaN,NaN


### Remove Null Price rows and change type to float

In [3]:
df = df[pd.to_numeric(df['Price'], errors='coerce').notnull()]

In [4]:
df = df.astype({"Price": float}, errors='raise')

### Remove duplicate ID

In [5]:
df['Immoweb ID'].value_counts()

9691876    4
9662650    3
9597143    3
8237073    3
9687272    3
          ..
9623733    1
9245768    1
9606238    1
9695276    1
7921699    1
Name: Immoweb ID, Length: 13893, dtype: int64

In [6]:
df = df.drop_duplicates(['Immoweb ID'], keep='last')
df['Immoweb ID'].value_counts()

9729720    1
9678715    1
9697071    1
9557609    1
9678634    1
          ..
9518030    1
9622950    1
9478102    1
9620790    1
7921699    1
Name: Immoweb ID, Length: 13893, dtype: int64

### Remove subtype 

In [7]:
df = df[df['property sub-type'] != "APARTMENT_BLOCK"]
df['property sub-type'].value_counts()

VILLA                   2468
GROUND_FLOOR            1606
DUPLEX                  1580
MIXED_USE_BUILDING      1240
PENTHOUSE               1170
FLAT_STUDIO              873
EXCEPTIONAL_PROPERTY     566
SERVICE_FLAT             500
MANSION                  479
TOWN_HOUSE               383
COUNTRY_COTTAGE          292
LOFT                     271
BUNGALOW                 212
FARMHOUSE                171
TRIPLEX                   99
KOT                       95
CHALET                    94
MANOR_HOUSE               62
CASTLE                    50
Name: property sub-type, dtype: int64

### Building condition

In [8]:
df['Building condition'].value_counts()
df['Building condition'].unique()

array(['Good', nan, 'As new', 'To renovate', 'To be done up',
       'Just renovated', 'To restore'], dtype=object)

In [9]:
building_condition_map = {'As new': 6, 'Just renovated': 5, 'Good': 4, 'To be done up': 3, 'To renovate':2, 'To restore':1}
df = df.applymap(lambda s: building_condition_map.get(s) if s in building_condition_map else s)

In [10]:
df['Building condition'].isnull().sum()

3828

In [11]:
df['Building condition'] = df['Building condition'].fillna(2)
df['Building condition'].isnull().sum()

0

### Living area

#### Fill missing values in Living area row

In [12]:
def fill_living_area(col):  
    if col['Living area'] > 0:
        return col['Living area']
    else: 
        if col['Property type'] == 'APARTMENT':
            return 95.0
        else:
            return 150.0


df['Living area'] = df.apply(lambda col: fill_living_area(col), axis=1)


In [13]:
# Test if there are null values in living area
df['Living area'].isnull().sum()

0

### One Hot Encoding of Property type, property sub-type & Post code

In [14]:
df = pd.get_dummies(df, columns=['Property type', 'property sub-type', 'Post code'])

### Kitchen type	


In [15]:
Kit_type_dict = {"USA uninstalled" : 0, 
                 "Not installed" : 0, 
                 "Installed": 1, 
                 "USA installed": 1,
                 "Semi equipped": 1,
                 "USA semi equipped": 1,
                 "Hyper equipped": 2,
                 "USA hyper equipped": 2
                }

df = df.replace(Kit_type_dict)
df["Kitchen type"] = df["Kitchen type"].fillna(0)


df

,Immoweb ID,Price,Building condition,Kitchen type,Bedrooms,Furnished,Terrace surface,Tenement building,Number of frontages,Swimming pool,...,Post code_9960,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992
0,9729720,179000.0,4.0,1.0,1.0,Yes,9.0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,9729785,255000.0,4.0,1.0,1.0,Yes,46.0,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,9729784,255000.0,4.0,1.0,1.0,Yes,46.0,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,9729780,620000.0,4.0,1.0,3.0,Yes,70.0,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,9727201,379000.0,4.0,2.0,2.0,No,110.0,No,3.0,No,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,132000.0,6.0,0.0,1.0,NaN,NaN,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
14537,7770553,177000.0,6.0,0.0,1.0,NaN,NaN,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
14538,7770551,190000.0,6.0,0.0,2.0,NaN,NaN,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
14539,7921693,1350000.0,2.0,0.0,30.0,NaN,NaN,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


### Furnished


In [16]:
df['Furnished'].unique()

array(['Yes', 'No', nan], dtype=object)

In [17]:
df['Furnished'].isnull().sum()

4080

#### fill No to missing value then map to 1,0

In [18]:
df['Furnished'] = df['Furnished'].fillna("No")
df['Furnished'] = df['Furnished'].apply(lambda v: 0 if v == "No" else 1)

#### double check if value is 1, 0 and no missing value

In [19]:
print(df['Furnished'].unique())
print(df['Furnished'].isna().sum())

[1 0]
0


### Bedrooms


In [20]:
# Fill missing values with 2 bedrooms
df['Bedrooms'] = df['Bedrooms'].fillna(2).astype(int)

df["Bedrooms"].value_counts()

2      4389
3      2929
4      1723
1      1350
5       909
6       429
7       144
8       126
10       61
9        49
12       21
11       20
14       11
13        8
18        7
16        5
17        5
15        5
30        4
25        3
23        3
50        2
60        1
204       1
20        1
22        1
28        1
27        1
35        1
24        1
Name: Bedrooms, dtype: int64

### Terrace surface + Terrace	


#### Combine terrace & terrace surface in yes and no

In [21]:
df['Terrace_Combined'] = np.where(df['Terrace surface'].isnull() & df['Terrace'].isnull(), 'No', 'Yes')

#### Transform Terrace_Combined to 0 & 1

In [22]:
terrace = ['No', 'Yes']

ordenc = OrdinalEncoder(categories=[terrace])

df[["Terrace_Combined"]] = ordenc.fit_transform(df[["Terrace_Combined"]])

df

,Immoweb ID,Price,Building condition,Kitchen type,Bedrooms,Furnished,Terrace surface,Tenement building,Number of frontages,Swimming pool,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,179000.0,4.0,1.0,1,1,9.0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,255000.0,4.0,1.0,1,1,46.0,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,255000.0,4.0,1.0,1,1,46.0,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,620000.0,4.0,1.0,3,1,70.0,No,2.0,NaN,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,379000.0,4.0,2.0,2,0,110.0,No,3.0,No,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,132000.0,6.0,0.0,1,0,NaN,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,177000.0,6.0,0.0,1,0,NaN,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,190000.0,6.0,0.0,2,0,NaN,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,1350000.0,2.0,0.0,30,0,NaN,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


#### Drop Terrace and Terrace Surface columns

In [23]:
df = df.drop(columns =['Terrace', 'Terrace surface'])

df

,Immoweb ID,Price,Building condition,Kitchen type,Bedrooms,Furnished,Tenement building,Number of frontages,Swimming pool,How many fireplaces?,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,179000.0,4.0,1.0,1,1,No,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,255000.0,4.0,1.0,1,1,No,2.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,255000.0,4.0,1.0,1,1,No,2.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,620000.0,4.0,1.0,3,1,No,2.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,379000.0,4.0,2.0,2,0,No,3.0,No,NaN,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,132000.0,6.0,0.0,1,0,No,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,177000.0,6.0,0.0,1,0,No,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,190000.0,6.0,0.0,2,0,No,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,1350000.0,2.0,0.0,30,0,No,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


### ~~Tenement building	~~


In [24]:
df = df.drop(columns =['Tenement building'])

df

,Immoweb ID,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,How many fireplaces?,Garden,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,179000.0,4.0,1.0,1,1,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,255000.0,4.0,1.0,1,1,2.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,255000.0,4.0,1.0,1,1,2.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,620000.0,4.0,1.0,3,1,2.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,379000.0,4.0,2.0,2,0,3.0,No,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,132000.0,6.0,0.0,1,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,177000.0,6.0,0.0,1,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,190000.0,6.0,0.0,2,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,1350000.0,2.0,0.0,30,0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


### Swimming pool	


In [25]:
# Fill missing values with value 0
df['Swimming pool'].fillna(0, inplace = True)
df['Swimming pool'] = df['Swimming pool'].apply(lambda v: 0 if v == "No" else 1)
df['Swimming pool'].isnull().sum()

0

In [26]:
df['Swimming pool'].unique()

array([1, 0])

### Garden + Garden surface	


In [27]:
print(df['Garden'].isna().sum())
print(df['Garden'].unique())

11051
[nan 'Yes']


In [28]:
df.loc[df['Garden surface'].isna()]

,Immoweb ID,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,How many fireplaces?,Garden,...,Post code_9968,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined
0,9729720,179000.0,4.0,1.0,1,1,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
1,9729785,255000.0,4.0,1.0,1,1,2.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
2,9729784,255000.0,4.0,1.0,1,1,2.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
3,9729780,620000.0,4.0,1.0,3,1,2.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
4,9727201,379000.0,4.0,2.0,2,0,3.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,132000.0,6.0,0.0,1,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
14537,7770553,177000.0,6.0,0.0,1,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14538,7770551,190000.0,6.0,0.0,2,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1.0
14539,7921693,1350000.0,2.0,0.0,30,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0


#### Combine ['Garden'] and ['Garden surface] as ['garden_label] and map to 1,0

In [29]:
def categorise(col):  
    if col['Garden'] == "Yes" or col['Garden surface'] > 0:
        return 1
    return 0


df['garden_label'] = df.apply(lambda col: categorise(col), axis=1)

#### double check if value is 1, 0 and no missing value

In [30]:
df['garden_label'].unique()

array([0, 1])

#### drop column Garden and Garden surface

In [31]:
df = df.drop(columns =['Garden', 'Garden surface'])

df

,Immoweb ID,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,How many fireplaces?,Surface of the plot,...,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label
0,9729720,179000.0,4.0,1.0,1,1,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
1,9729785,255000.0,4.0,1.0,1,1,2.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
2,9729784,255000.0,4.0,1.0,1,1,2.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
3,9729780,620000.0,4.0,1.0,3,1,2.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
4,9727201,379000.0,4.0,2.0,2,0,3.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,132000.0,6.0,0.0,1,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0
14537,7770553,177000.0,6.0,0.0,1,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14538,7770551,190000.0,6.0,0.0,2,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14539,7921693,1350000.0,2.0,0.0,30,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0


### ~~Garden orientation~~


#### Drop garden orientation

In [32]:
df = df.drop(columns =['Garden orientation'])

df

,Immoweb ID,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,How many fireplaces?,Surface of the plot,...,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label
0,9729720,179000.0,4.0,1.0,1,1,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
1,9729785,255000.0,4.0,1.0,1,1,2.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
2,9729784,255000.0,4.0,1.0,1,1,2.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
3,9729780,620000.0,4.0,1.0,3,1,2.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
4,9727201,379000.0,4.0,2.0,2,0,3.0,0,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,132000.0,6.0,0.0,1,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0
14537,7770553,177000.0,6.0,0.0,1,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14538,7770551,190000.0,6.0,0.0,2,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1.0,0
14539,7921693,1350000.0,2.0,0.0,30,0,NaN,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0.0,0


### ~~How many fireplaces?	~~


#### Discard fireplaces

In [33]:
df = df.drop(columns =["How many fireplaces?"])

df

,Immoweb ID,Price,Building condition,Kitchen type,Bedrooms,Furnished,Number of frontages,Swimming pool,Surface of the plot,Living area,...,Post code_9970,Post code_9971,Post code_9981,Post code_9982,Post code_9988,Post code_9990,Post code_9991,Post code_9992,Terrace_Combined,garden_label
0,9729720,179000.0,4.0,1.0,1,1,NaN,1,NaN,63.0,...,0,0,0,0,0,0,0,0,1.0,0
1,9729785,255000.0,4.0,1.0,1,1,2.0,1,NaN,42.0,...,0,0,0,0,0,0,0,0,1.0,0
2,9729784,255000.0,4.0,1.0,1,1,2.0,1,NaN,42.0,...,0,0,0,0,0,0,0,0,1.0,0
3,9729780,620000.0,4.0,1.0,3,1,2.0,1,NaN,150.0,...,0,0,0,0,0,0,0,0,1.0,0
4,9727201,379000.0,4.0,2.0,2,0,3.0,0,NaN,120.0,...,0,0,0,0,0,0,0,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,7770552,132000.0,6.0,0.0,1,0,NaN,1,NaN,95.0,...,0,0,0,0,0,0,0,0,0.0,0
14537,7770553,177000.0,6.0,0.0,1,0,NaN,1,NaN,95.0,...,0,0,0,0,0,0,0,0,1.0,0
14538,7770551,190000.0,6.0,0.0,2,0,NaN,1,NaN,95.0,...,0,0,0,0,0,0,0,0,1.0,0
14539,7921693,1350000.0,2.0,0.0,30,0,NaN,1,NaN,1500.0,...,0,0,0,0,0,0,0,0,0.0,0


### Surface of the plot




In [34]:
# Fill empty values with 0
df['Surface of the plot'].fillna(0, inplace = True)

### Frontages

In [35]:
df['Number of frontages'].isnull().sum()

3927

In [36]:
#get ['number of frontages'] with values and calc mean
selected_rows = df[~df['Number of frontages'].isnull()]
mean_num_of_frontages = selected_rows['Number of frontages'].mean(axis=0).round(0)
mean_num_of_frontages

3.0

In [37]:
# fill mean value to missing value
df['Number of frontages'] = df['Number of frontages'].fillna(mean_num_of_frontages)
df['Number of frontages'].isnull().sum()

0

In [38]:
df['Number of frontages'] = df['Number of frontages'].astype(int)

In [39]:
# double check
df['Number of frontages'].value_counts()

3     5260
4     3504
2     3346
1       91
6        6
7        2
5        1
16       1
Name: Number of frontages, dtype: int64

### Final check missing value %




In [40]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Immoweb ID,Immoweb ID,0.0
Price,Price,0.0
Building condition,Building condition,0.0
Kitchen type,Kitchen type,0.0
Bedrooms,Bedrooms,0.0
...,...,...
Post code_9990,Post code_9990,0.0
Post code_9991,Post code_9991,0.0
Post code_9992,Post code_9992,0.0
Terrace_Combined,Terrace_Combined,0.0


In [41]:
df.to_csv('data/clean_data.csv', index=False)